In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import json
import time
from dotenv import load_dotenv

In [2]:
user_train_path = '/home/grads/m/mbismay/llm-based-seq-rec/beauty/beauty_data/processed_data/user_train.json'
user_valid_path = '/home/grads/m/mbismay/llm-based-seq-rec/beauty/beauty_data/processed_data/user_valid.json'
user_test_path = '/home/grads/m/mbismay/llm-based-seq-rec/beauty/beauty_data/processed_data/user_test.json'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)

In [4]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[7, 8, 9, 9, 10]
[7, 8, 9, 9, 10, 11]
[8, 9, 9, 10, 11, 12]


In [5]:
item_information_path = '/home/grads/m/mbismay/llm-based-seq-rec/beauty/beauty_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

1216


In [6]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


In [7]:
reasoning_train_data = "./reasoning_dataset/reasoning_data/reasoning_train_dict.pkl"
reasoning_valid_data = "./reasoning_dataset/reasoning_data/reasoning_valid_dict.pkl"
reasoning_test_data = "./reasoning_dataset/reasoning_data/reasoning_test_dict.pkl"

In [8]:
with open(reasoning_train_data, 'rb') as f:
    reasoning_train = pickle.load(f)
with open(reasoning_valid_data, 'rb') as f:
    reasoning_valid = pickle.load(f)
with open(reasoning_test_data, 'rb') as f:
    reasoning_test = pickle.load(f)

In [9]:
# print(reasoning_prompt_train[user_id])
print(reasoning_train[user_id])
print(reasoning_valid[user_id])
print(reasoning_test[user_id])


        1. The user profile shows that the user values durability and quality in their purchases. The user dislikes products that do not meet expectations, as seen with the 100% Cotton Premium Salon Towels. Despite being soft and thick, the towels unraveled after the first wash, which was disappointing for the user. The user seems to value durability and quality in their purchases.
        The user likes products that are long-lasting, good value for money, and have a lovely scent. The user dislikes products that have a weak scent or are drying to the skin.
        2. The user will like the Utopia Care Stainless Steel Fingernail Clipper because it is a high-quality product made of stainless steel, which is a durable material that will last a long time. The clipper has a unique shape and style, which the user values in their purchases. Although the clipper is larger than typical and has a swivel that can come out, it works great and resembles toenail clippers. However, the user values 

In [28]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    if 'brand' in item_information:
        item_details += f" Brand: {item_information['brand']}"
    if 'price' in item_information:
        item_details += f" Price: {item_information['price']}"
    if 'description' in item_information:
        item_details += f" Description: {item_information['description']}"
    return item_details

In [29]:
user_profile_summary_path = '../beauty_data/processed_data/user_profile_summary.json'
with open(user_profile_summary_path, 'r') as f:
    user_profile_summary_dict = json.load(f)
print(len(user_profile_summary_dict), user_profile_summary_dict['1'])

160 

The user dislikes products that do not meet expectations, as seen with the 100% Cotton Premium Salon Towels. Despite being soft and thick, the towels unraveled after the first wash, which was disappointing for the user. The user seems to value durability and quality in their purchases.


In [30]:
def get_new_reasoning_text(reasoning):
    # print(f"reasoning: {reasoning}")
    new_reasoning_text = f"Prediction: {reasoning.split(' ')[-1]} \n"
    new_reasoning_text += ' '.join(s.strip() for s in reasoning.split("\n")[:-1])
    # print(f"new_reasoning_text: {new_reasoning_text}")
    return new_reasoning_text

In [31]:
### For Train Data
final_train_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_train.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    if user in user_profile_summary_dict:
        temp_dict["instruction"] = "You are an expert beauty product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = f"""User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        temp_dict["instruction"] = "You are an expert beauty product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = "User purchased the following items in the given order. List of recent items and their description -\n"
    
    item_list = user_train_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user will purchase the next target item - {target_item_details}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{item_information_dict[str(target_item)]['title']}" in atmost 100 words."""
    temp_dict["input"] = prompt

    temp_dict["output"] = get_new_reasoning_text(reasoning)
    final_train_dict.append(temp_dict)
    # print(final_train_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_train_dict:", len(final_train_dict), final_train_dict[0])
print("Avg Length:", avg_total_len/len(final_train_dict))
print("Max length:", max_total_len)

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 274 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0
Length of final_train_dict: 616 {'instruction': "You are an expert beauty product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User Profile - \n\nThe user seems to prefer hypoallergenic fragrances and natural ingredients. They liked Fresh Eau De Parfum EDP - Fig Apricot, which is a hypoallergenic fragrance, indicating their preference for pleasant scents that don\'t cause allergies. They also liked

In [32]:
final_train_dict[183]

{'instruction': "You are an expert beauty product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ",
 'input': 'User purchased the following items in the given order. List of recent items and their description -\nLiked Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz Brand: Yardley Price:  Description: \n\nProduct: Shower Gel\nPositive features: Lovely holiday scent, mild scrub, good quality, great smell, works well, fresh fruity scent, creamy suds.\nNegative features: Small size, discontinued.\n\nConcise product description: "Shower Gel with\nLiked Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz. Brand: Fruits & Passion Price:  Description: \n\nThe product is a Body Shop shower gel, with a strong fragrance and great quality, but may be disco

In [33]:
yes_cnt = 0
no_cnt = 0
for train_data in final_train_dict:
    pred = train_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

568 48 616


In [38]:
cnt = 0
cleaned_final_train_dict = []
for temp_dict in final_train_dict:
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    if total_len>800:
        cnt += 1
        continue
    cleaned_final_train_dict.append(temp_dict)

print(cnt)
print(len(final_train_dict), len(cleaned_final_train_dict))

9
616 607


In [39]:
yes_cnt = 0
no_cnt = 0
for train_data in cleaned_final_train_dict:
    pred = train_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

560 47 607


In [41]:
final_train_path = "./final_data/beauty/train.json"
with open(final_train_path, 'w') as f:
    json.dump(cleaned_final_train_dict, f)

In [42]:
### For Valid Data
final_valid_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_valid.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    if user in user_profile_summary_dict:
        temp_dict["instruction"] = "You are an expert beauty product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = f"""User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        temp_dict["instruction"] = "You are an expert beauty product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = "User purchased the following items in the given order. List of recent items and their description -\n"
    
    item_list = user_train_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user will purchase the next target item - {target_item_details}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{item_information_dict[str(target_item)]['title']}" in atmost 100 words."""
    temp_dict["input"] = prompt

    temp_dict["output"] = get_new_reasoning_text(reasoning)
    final_valid_dict.append(temp_dict)
    # print(final_valid_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_valid_dict:", len(final_valid_dict), final_valid_dict[0])
print("Avg Length:", avg_total_len/len(final_valid_dict))
print("Max length:", max_total_len)

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 274 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0
Length of final_valid_dict: 616 {'instruction': "You are an expert beauty product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User Profile - \n\nThe user seems to prefer hypoallergenic fragrances and natural ingredients. They liked Fresh Eau De Parfum EDP - Fig Apricot, which is a hypoallergenic fragrance, indicating their preference for pleasant scents that don\'t cause allergies. They also liked

In [43]:
yes_cnt = 0
no_cnt = 0
for valid_data in final_valid_dict:
    pred = valid_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

573 43 616


In [44]:
cnt = 0
cleaned_final_valid_dict = []
for temp_dict in final_valid_dict:
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    if total_len>800:
        cnt += 1
        continue
    cleaned_final_valid_dict.append(temp_dict)

print(cnt)
print(len(final_valid_dict), len(cleaned_final_valid_dict))

13
616 603


In [45]:
yes_cnt = 0
no_cnt = 0
for valid_data in cleaned_final_valid_dict:
    pred = valid_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

560 43 603


In [51]:
final_valid_path = "./final_data/beauty/valid.json"
with open(final_valid_path, 'w') as f:
    json.dump(cleaned_final_valid_dict, f)

In [47]:
### For Test Data
final_test_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, reasoning in reasoning_test.items():
    # print(f"User: {user}")
    cnt += 1
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)

    temp_dict = dict()
    if user in user_profile_summary_dict:
        temp_dict["instruction"] = "You are an expert beauty product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = f"""User Profile - {user_profile_summary_dict[user]}
        User purchased the following items in the given order. List of recent items and their description -
        """
    else:
        temp_dict["instruction"] = "You are an expert beauty product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. "
        prompt = "User purchased the following items in the given order. List of recent items and their description -\n"
    
    item_list = user_train_dict[user]
    # print(item_list)
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    target_item_rating = user_rating_dict[user_rating_dict['item'] == target_item]['rating'].values[0]
    prompt += f"\nThe user will purchase the next target item - {target_item_details}"
    
    prompt += f"""\nDo not use any information not mentioned above. As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{item_information_dict[str(target_item)]['title']}" in atmost 100 words."""
    temp_dict["input"] = prompt

    temp_dict["output"] = get_new_reasoning_text(reasoning)
    final_test_dict.append(temp_dict)
    # print(final_test_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # break
print("Length of final_test_dict:", len(final_test_dict), final_test_dict[0])
print("Avg Length:", avg_total_len/len(final_test_dict))
print("Max length:", max_total_len)

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 274 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0
Length of final_test_dict: 616 {'instruction': "You are an expert beauty product recommender. Given a user's profile and list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User Profile - \n\nThe user seems to prefer hypoallergenic fragrances and natural ingredients. They liked Fresh Eau De Parfum EDP - Fig Apricot, which is a hypoallergenic fragrance, indicating their preference for pleasant scents that don\'t cause allergies. They also liked 

In [48]:
yes_cnt = 0
no_cnt = 0
for test_data in final_test_dict:
    pred = test_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', test_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

571 45 616


In [49]:
cnt = 0
cleaned_final_test_dict = []
for temp_dict in final_test_dict:
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    if total_len>800:
        cnt += 1
        continue
    cleaned_final_test_dict.append(temp_dict)

print(cnt)
print(len(final_test_dict), len(cleaned_final_test_dict))

12
616 604


In [50]:
yes_cnt = 0
no_cnt = 0
for test_data in cleaned_final_test_dict:
    pred = test_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', test_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

560 44 604


In [55]:
print(len(final_test_dict))

616


In [54]:
user = 121
print(final_train_dict[user])
print(final_valid_dict[user])
print(final_test_dict[user])

{'instruction': "You are an expert beauty product recommender. Given a user's list of recent liked and disliked items and their price, brand, and description, identify whether the user will like or dislike the target item by providing reasoning for the same. Please follow further instruction below. ", 'input': 'User purchased the following items in the given order. List of recent items and their description -\nLiked Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz Brand: Yardley Price:  Description: \n\nProduct: Shower Gel\nPositive features: Lovely holiday scent, mild scrub, good quality, great smell, works well, fresh fruity scent, creamy suds.\nNegative features: Small size, discontinued.\n\nConcise product description: "Shower Gel with\nLiked Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz. Brand: Fruits & Passion Price:  Description: \n\nThe product is a Body Shop shower gel, with a strong fragrance and great quality, but may be discon

In [53]:
final_test_path = "./final_data/beauty/test.json"
with open(final_test_path, 'w') as f:
    json.dump(cleaned_final_test_dict, f)